# A modular GCD algorithm: Overview of algorithm

The individual steps are the same as in K. Weber et al. / Journal of Algorithms 54 (2005) 152–167. Given are two integers $a, b \in \mathbb{Z}^*$. This notebook serves as an overview of the algorithm.

In [1]:
a = 23 * 87
b = 23 * 55
a, b

(2001, 1265)

In [2]:
if a >= b:
    U = a
    V = b
else:
    U = b
    V = a

## MGCD1: [Find suitable moduli]

$$
\begin{align}
1 \quad & n = \lfloor \text{lg} U \rfloor + 1 \\
2 \quad & w = \text{ an integer satisfying }\pi(2^w) - \pi(2^{w-1}) \leq \text{max}\lbrace \lceil 2^{w/2}+n \rceil, 9 \rbrace \\
3 \quad & Q = \text{ the set of }\lceil 2^{w/2} + n \rceil\text{ largest primes less than }2 w
\end{align}
$$

In [3]:
n = floor(log(U, 2)) + 1
n

11

In [4]:
def pi(x):
    """
    Calculates the number of primes less than or equal to the given real number
    
    Parameters
    ----------
    x: real
        A real number.
        
    Returns
    -------
    pi: integer
        Number of primes.
    """
    P = Primes()
    i = 0
    while P.unrank(i) <= x:
        i += 1

    return i

In [5]:
w = 1
while pi(2^w) - pi(2^(w-1)) < max(ceil(2^(w/2) + n), 9):
    w += 1

w

9

In [6]:
def getLargestPrimes(number, bound):
    primes = set()
    P = Primes()
    i = 0
    while P.unrank(i) < bound:
        i += 1

    while len(primes) < number:
        assert(i > 0)
        i -= 1
        primes.add(P.unrank(i))

    return primes

In [7]:
Q = getLargestPrimes(ceil(2^(w/2) + n), 2^w)
Q

{311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

## MGCD2: [Convert to modular representation]

$$
\begin{align}
1 \quad & \text{For all }q ∈ \mathcal{Q}\text{ do }[u_q, v_q ] = [U\text{ mod } q, V\text{ mod }q]
\end{align}
$$

In [8]:
u, v = {}, {}
for q in Q:
    u[q], v[q] = U % q, V % q
    if u[q] > q / 2:
        u[q] -= q

    if v[q] > q / 2:
        v[q] -= q

u, v

({311: 135,
  313: 123,
  317: 99,
  331: 15,
  337: -21,
  347: -81,
  349: -93,
  353: -117,
  359: -153,
  367: 166,
  373: 136,
  379: 106,
  383: 86,
  389: 56,
  397: 16,
  401: -4,
  409: -44,
  419: -94,
  421: -104,
  431: -154,
  433: -164,
  439: -194,
  443: -214,
  449: 205,
  457: 173,
  461: 157,
  463: 149,
  467: 133,
  479: 85,
  487: 53,
  491: 37,
  499: 5,
  503: -11,
  509: -35},
 {311: 21,
  313: 13,
  317: -3,
  331: -59,
  337: -83,
  347: -123,
  349: -131,
  353: -147,
  359: -171,
  367: 164,
  373: 146,
  379: 128,
  383: 116,
  389: 98,
  397: 74,
  401: 62,
  409: 38,
  419: 8,
  421: 2,
  431: -28,
  433: -34,
  439: -52,
  443: -64,
  449: -82,
  457: -106,
  461: -118,
  463: -124,
  467: -136,
  479: -172,
  487: -196,
  491: -208,
  499: -232,
  503: -244,
  509: 247})

## MGCD3: [Reduction loop]

$$
\begin{align}
1 \quad & \text{Repeat} \\
2 \quad & \quad \mathcal{P} = \lbrace q ∈ \mathcal{Q} | V\text{ mod }q \neq 0 \rbrace \\
3 \quad & \quad p =\text{ an element of }\mathcal{P}\text{ for which }|u_p / v_p\text{ mod }p|\text{ is minimal.} \\
4 \quad & \quad b = u_p / v_p\text{ mod }p \\
5 \quad & \quad Q = Q - \lbrace p \rbrace \\
6 \quad & \quad \text{For all }q ∈ \mathcal{Q}\text{ do }[u_q, v_q] = [v_q, (u_q - b v_q )/p\text{ mod }q] \\
7 \quad & \text{Until }∀q ∈ \mathcal{Q}, v_q = 0
\end{align}
$$

In [15]:
P = set()
for q in Q:
    if V % q != 0:
        P.add(q)

P

{311,
 313,
 317,
 331,
 337,
 347,
 349,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

In [16]:
def findPrime(P):
    minimum = None
    current_p = None
    for p in P:
        abs_module = abs(u[p] / v[p] % p)
        if minimum == None or abs_module < minimum:
            minimum = abs_module
            current_p = p
    
    return current_p        

In [17]:
p = findPrime(P)
p

311

In [18]:
b = (u[p] / v[p]) % p
if b > p / 2:
    b -= p

b

-55

In [19]:
Q.remove(p)
Q

{313,
 317,
 331,
 337,
 347,
 349,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

In [20]:
del u[p], v[p]
for q in Q:
    u[q], v[q] = v[q], (u[q] - b * v[q]) / p % q
    if u[q] > q / 2:
        u[q] -= q

    if v[q] > q / 2:
        v[q] -= q

u, v

({313: -23,
  317: -23,
  331: -23,
  337: -23,
  347: -23,
  349: -23,
  359: -23,
  367: -23,
  373: -23,
  379: -23,
  383: -23,
  389: -23,
  397: -23,
  401: -23,
  409: -23,
  419: -23,
  421: -23,
  431: -23,
  433: -23,
  439: -23,
  443: -23,
  449: -23,
  457: -23,
  461: -23,
  463: -23,
  467: -23,
  479: -23,
  487: -23,
  491: -23,
  499: -23,
  503: -23,
  509: -23},
 {313: 0,
  317: 0,
  331: 0,
  337: 0,
  347: 0,
  349: 0,
  359: 0,
  367: 0,
  373: 0,
  379: 0,
  383: 0,
  389: 0,
  397: 0,
  401: 0,
  409: 0,
  419: 0,
  421: 0,
  431: 0,
  433: 0,
  439: 0,
  443: 0,
  449: 0,
  457: 0,
  461: 0,
  463: 0,
  467: 0,
  479: 0,
  487: 0,
  491: 0,
  499: 0,
  503: 0,
  509: 0})

## MGCD4: [Return standard representation]

$$
\begin{align}
1 \quad & G = 0 \\
2 \quad & \text{Repeat} \\
3 \quad & \quad p =\text{ an element of }\mathcal{Q} \\
4 \quad & \quad \mathcal{Q} = \mathcal{Q} - \lbrace p\rbrace \\
5 \quad & \quad G = u_p + p G \\
6 \quad & \quad \text{For all }q ∈ \mathcal{Q}\text{ do }u_q = (u_q - u_p) / p\text{ mod }q \\
7 \quad & \text{Until }∀q ∈ \mathcal{Q}, u_q = 0 \\
8 \quad & \text{Return }|G|
\end{align}
$$

In [21]:
G = 0

In [22]:
p = Q.pop()
p

389

In [23]:
Q

{313,
 317,
 331,
 337,
 347,
 349,
 359,
 367,
 373,
 379,
 383,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

In [24]:
G = u[p] + p * G
G

-23

In [25]:
for q in Q:
    u[q] = (u[q] - u[p]) / p % q
    if u[q] > q / 2:
        u[q] -= q

del u[p]
u

{313: 0,
 317: 0,
 331: 0,
 337: 0,
 347: 0,
 349: 0,
 359: 0,
 367: 0,
 373: 0,
 379: 0,
 383: 0,
 397: 0,
 401: 0,
 409: 0,
 419: 0,
 421: 0,
 431: 0,
 433: 0,
 439: 0,
 443: 0,
 449: 0,
 457: 0,
 461: 0,
 463: 0,
 467: 0,
 479: 0,
 487: 0,
 491: 0,
 499: 0,
 503: 0,
 509: 0}

In [26]:
abs(G)

23